In [1]:
import json
import pandas as pd
from tqdm import tqdm
from langchain_community.llms import Ollama

# Dataset

In [2]:
def read_json_array(path):
  data = []
  with open(path, 'r') as file:
    for line in file:
      json_object = json.loads(line.strip())
      data.append(json_object)
  return data

dataset_path = '../../datasets/amazon_reviews/'
data_path = dataset_path + "Musical_Instruments.json"

data = read_json_array(data_path)
df = pd.DataFrame(data)

In [3]:
amazon_reviews = pd.DataFrame()
amazon_reviews['Reviews'] = df['reviewText'].copy()
amazon_reviews['Ratings'] = df['overall'].copy()
amazon_reviews = amazon_reviews.sample(n=10000, random_state=42).reset_index(drop=True)

# Modeling

In [4]:
llm = Ollama(model="gemma2")

/var/folders/87/dhv__9sj0yv1kz0pfv2ds9nh0000gn/T/ipykernel_1875/2721138275.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma2")


### Step 1 - Creating Topics for batches of dataset

In [37]:
def generate_topics_in_batches(amazon_reviews, batch_size=100, llm=None):
    """
    Generate topics from Amazon reviews in batches to handle large datasets.
    
    Parameters:
    amazon_reviews (pd.DataFrame): DataFrame containing Amazon reviews
    batch_size (int): Number of reviews to process in each batch
    llm: The language model instance to use for generating topics
    
    Returns:
    list: List of lists containing topics for each batch
    """
    if llm is None:
        raise ValueError("LLM instance must be provided")
    
    all_batch_topics = []
    total_reviews = len(amazon_reviews)
    
    # Process reviews in batches
    for start_idx in tqdm(range(0, total_reviews, batch_size)):
        end_idx = min(start_idx + batch_size, total_reviews)
        batch_reviews = amazon_reviews.iloc[start_idx:end_idx]
        
        # Generate prompt for current batch
        prompt_labels_generator = f'''You are provided with amazon reviews and helping to cluster the reviews based on the topics.
Please create topics based on the reviews provided. Keep the topics general and not specific to the reviews. 
DO NOT include any specific details. NO TOPICS LIKE  DJ Equipment, Guitar, Piano, etc. Topics should be general and helpful for business steakholders to understand what people are thinking about the products,
Amazon Reviews: {batch_reviews['Reviews'].tolist()}
Please return only in CSV format with the following structure:
Topic1, Topic2, Topic3, Topic4, Topic5,...
Return only the topics in CSV format and nothing else.
'''

        # Get topics for current batch
        result = llm.invoke(prompt_labels_generator, temperature=0.0)
        
        # Convert CSV string to list of topics
        batch_topics = [topic.strip() for topic in result.split(',')]
        all_batch_topics.append(batch_topics)
        
        print(f"Processed batch {len(all_batch_topics)}: reviews {start_idx} to {end_idx}")
    
    return all_batch_topics

In [38]:
topics_by_batch = generate_topics_in_batches(amazon_reviews.sample(1000, random_state=42), batch_size=10, llm=llm)

  1%|          | 1/100 [00:04<08:06,  4.92s/it]

Processed batch 1: reviews 0 to 10


  2%|▏         | 2/100 [00:09<07:51,  4.81s/it]

Processed batch 2: reviews 10 to 20


  3%|▎         | 3/100 [00:13<07:23,  4.57s/it]

...

Processed batch 98: reviews 970 to 980


 99%|█████████▉| 99/100 [06:57<00:03,  3.62s/it]

Processed batch 99: reviews 980 to 990


100%|██████████| 100/100 [07:01<00:00,  4.21s/it]

Processed batch 100: reviews 990 to 1000


In [39]:
all_unique_topics = list(set([topic for batch in topics_by_batch for topic in batch]))

In [41]:
all_unique_topics

['Technical Aspects',
 'User Experience',
 'Sound & Performance',
 'Guitar Straps',
 'Sound/Performance',
 'Aesthetics',
 'Guitar Effects Pedals',
 'Impedance Matching',
 'Customer Experience',
 'Product Durability',
 'Performance & Functionality',
 'Price Value',
 'Durability & Performance',
 'Features',
 'Value',
 'Comfort',
 'Shipping & Delivery',
 'Microphone Sensitivity',
 'Quality',
 'Learning Curve',
 'Size',
 'Usage Experience',
 'Vocal Harmonizers',
 'Product Quality',
 'Size & Portability',
 'Value for Money',
 'Price',
 'Performance Issues',
 'Guitar Amplifiers',
 'Usage',
 'Aesthetics & Comfort',
 'Performance',
 'Delivery Experience',
 'Customer Satisfaction',
 'Installation & Setup',
 'Target Audience',
 'Comfort/Design',
 'Fit & Functionality',
 'Brand Perception',
 'Experience',
 'Cello Pickup Installation',
 'Shipping Experience',
 'Compatibility',
 'Customer Service',
 'Product Performance',
 'Usability',
 'Quality & Durability',
 'Performance Quality',
 'Shipping & P

### Step 2 - Combining all topics to a smaller more general subset

In [40]:
prompt_merge_topics = f'''You are provided with topics generated from Amazon reviews.
Please merge the topics into a smaller number of topics. The topics should be VERY GENERAL and NOT specific. Do not output more than 15 topics. 
Merge/combine topics that are similar or related.
Topics: {all_unique_topics}
Please return only in CSV format with the following structure:
MergedTopic1, MergedTopic2, MergedTopic3, MergedTopic4, MergedTopic5,...
Return only the merged topics in CSV format and nothing else.
'''

# Get merged topics
proper_topics_str = llm.invoke(prompt_merge_topics, temperature=0.0)

In [43]:
proper_topics = [topic.strip() for topic in proper_topics_str.split(',')]
print(proper_topics)

['Technical Aspects', 'User Experience', 'Sound & Performance', 'Aesthetics', 'Durability', 'Price & Value', 'Customer Experience', 'Size & Portability', 'Functionality & Usability', 'Shipping & Delivery', 'Brand Perception', 'Product Quality', 'Comfort & Design', 'Learning Curve', 'Target Audience', 'Usage Experience', 'Gift Giving', 'Emotional Response', 'Compatibility', 'Customer Service']


In [5]:
# For clean run (without rerunning previous code)
proper_topics = ['Technical Aspects', 'User Experience', 'Sound & Performance', 'Aesthetics', 'Durability', 'Price & Value', 'Customer Experience', 'Size & Portability', 'Functionality & Usability', 'Shipping & Delivery', 'Brand Perception', 'Product Quality', 'Comfort & Design', 'Learning Curve', 'Target Audience', 'Usage Experience', 'Gift Giving', 'Emotional Response', 'Compatibility', 'Customer Service']

### Step 3 - Assigning batches of dataset to Topics Generated in Step 2 and sentiment of the review

In [25]:
def assign_topics_in_batches(input_df, topics, batch_size, llm=None):
    """
    Assign topics and sentiment to Amazon reviews in batches and update DataFrame directly.

    Parameters:
    input_df (pd.DataFrame): DataFrame containing Amazon reviews
    topics (str): String of topics to assign from
    batch_size (int): Number of reviews to process in each batch
    llm: The language model instance to use for assigning topics

    Returns:
    pd.DataFrame: Updated DataFrame with topic and sentiment assignments
    """
    if llm is None:
        raise ValueError("LLM instance must be provided")

    # Create a copy of the DataFrame to avoid modifying the original
    df = input_df.copy()

    # Initialize Topic and Sentiment columns with 'Unknown'
    df['Topic'] = 'Unknown'
    df['Sentiment'] = 'Unknown'

    total_reviews = len(df)

    # Process reviews in batches with progress bar
    for start_idx in tqdm(range(0, total_reviews, batch_size), desc="Assigning topics"):
        end_idx = min(start_idx + batch_size, total_reviews)
        batch_reviews_list = df.iloc[start_idx:end_idx]
        batch_reviews = " ".join([f"Comment {i + 1}: {review}," for i, review in enumerate(batch_reviews_list['Reviews'])])
        
        # Generate prompt for current batch
        prompt_assigning_prompt = f'''You are provided with amazon reviews and helping to cluster the reviews based on the topics.
Please thoroghly read the amazon review below and assign the proper topic (from the list of topics) and sentiment (Positive, Negative or Neutral) to the review.
Amazon Review: {batch_reviews}
Topics: {topics}
Please return in CSV format only one topic and one sentiment for respective review and nothing else. So only 1 coma. Do not use triple backtick blocks. Only output exactly as on the example below:
Example: Some happy review from customer.
Output: Selected Topic, Positive
'''

        # Get assignments for current batch
        result = llm.invoke(prompt_assigning_prompt, temperature=0.0)
        try:
            topic, sentiment = [topic.strip() for topic in result.split(',')]
            current_idx = start_idx
            df.at[current_idx, 'Topic'] = topic
            df.at[current_idx, 'Sentiment'] = sentiment
        except Exception as e:
            print(f"\nUnexpected error processing batch starting at index {start_idx}: {str(e)}")
            continue

    return df


In [26]:
amazon_reviews_with_topics = assign_topics_in_batches(
    amazon_reviews,
    topics=str(proper_topics),
    batch_size=1,
    llm=llm
)

Assigning topics:   2%|▏         | 240/10000 [05:15<3:51:29,  1.42s/it]


Unexpected error processing batch starting at index 239: not enough values to unpack (expected 2, got 1)


Assigning topics:   2%|▏         | 248/10000 [05:27<4:23:40,  1.62s/it]


Unexpected error processing batch starting at index 247: too many values to unpack (expected 2)


Assigning topics:   3%|▎         | 300/10000 [06:37<5:30:18,  2.04s/it]

...

Assigning topics:  99%|█████████▉| 9942/10000 [3:43:29<01:31,  1.58s/it]


Unexpected error processing batch starting at index 9941: too many values to unpack (expected 2)


Assigning topics: 100%|█████████▉| 9964/10000 [3:44:01<01:00,  1.69s/it]


Unexpected error processing batch starting at index 9963: too many values to unpack (expected 2)


Assigning topics: 100%|██████████| 10000/10000 [3:45:00<00:00,  1.35s/it]


In [27]:
amazon_reviews_with_topics.to_csv('outputs/amazon_reviews_with_topics_10000.csv', index=False)

In [28]:
amazon_reviews_with_topics['Topic'].value_counts()

Topic
Sound & Performance          2156
Functionality & Usability    1775
Durability                   1102
Product Quality              1017
Price & Value                 753
Aesthetics                    435
Usage Experience              419
User Experience               362
Customer Experience           311
Size & Portability            306
Technical Aspects             299
Compatibility                 245
Gift Giving                   199
Shipping & Delivery           188
Comfort & Design              152
Unknown                       109
Learning Curve                104
Target Audience                33
Emotional Response             18
Brand Perception               13
Customer Service                3
Battery Life                    1
Name: count, dtype: int64

In [35]:
amazon_reviews_with_topics = pd.read_csv('outputs/amazon_reviews_with_topics_10000.csv')

In [42]:
def resolve_unkown(df, topics, llm=None):
    """
    Resolve 'Unknown' topics and sentiments in Amazon reviews DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing Amazon reviews with 'Unknown' topics and sentiments
    llm: The language model instance to use for resolving topics and sentiments

    Returns:
    pd.DataFrame: Updated DataFrame with resolved topics and sentiments
    """
    if llm is None:
        raise ValueError("LLM instance must be provided")

    # Create a copy of the DataFrame to avoid modifying the original
    df_resolved = df.copy()

    # Get indices of rows with 'Unknown' topics and sentiments
    unknown_indices = df_resolved[(df_resolved['Topic'] == 'Unknown') | (df_resolved['Sentiment'] == 'Unknown')].index

    # Resolve 'Unknown' topics and sentiments
    for idx in tqdm(unknown_indices, desc="Resolving Unknowns"):
        review = df_resolved.loc[idx, 'Reviews']
        
        # Generate prompt for current review
        prompt_assigning_prompt = f'''You are provided with amazon reviews and helping to classify the reviews based on the topics.
Please thoroghly read the amazon review below and assign the proper topic (from the list of topics) and sentiment (Positive, Negative or Neutral) to the review.
List of Topics: {topics}
Amazon Review: {review}
Please return in CSV format only one topic and one sentiment for respective review and nothing else. So only 1 coma. Do not use triple backtick blocks. Only output exactly as on the example below:
Example: Some happy review from customer.
Output: Selected Topic, Positive
'''
        # Get assignments for current review
        result = llm.invoke(prompt_assigning_prompt, temperature=0.0)
        try:
            topic, sentiment = [topic.strip() for topic in result.split(',')]
            df_resolved.at[idx, 'Topic'] = topic
            df_resolved.at[idx, 'Sentiment'] = sentiment
        except Exception as e:
            print(f"\nUnexpected error processing review at index {idx}: {str(e)}")
            continue
    return df_resolved

Resolving errors (Usually it's empty rows)

In [43]:
# Resolving
amazon_reviews_resolved = resolve_unkown(amazon_reviews_with_topics, topics=proper_topics, llm=llm)

Resolving Unknowns:   1%|          | 1/109 [00:01<02:16,  1.26s/it]


Unexpected error processing review at index 239: not enough values to unpack (expected 2, got 1)


Resolving Unknowns:   5%|▍         | 5/109 [00:05<01:49,  1.05s/it]


Unexpected error processing review at index 591: not enough values to unpack (expected 2, got 1)


Resolving Unknowns:  10%|█         | 11/109 [00:14<01:58,  1.21s/it]

...

Resolving Unknowns:  40%|████      | 44/109 [01:00<01:23,  1.28s/it]


Unexpected error processing review at index 4343: too many values to unpack (expected 2)


Resolving Unknowns:  93%|█████████▎| 101/109 [02:03<00:08,  1.08s/it]


Unexpected error processing review at index 9568: not enough values to unpack (expected 2, got 1)


Resolving Unknowns: 100%|██████████| 109/109 [02:10<00:00,  1.20s/it]


In [45]:
amazon_reviews_resolved['Sentiment'].value_counts()

Sentiment
Positive               7295
Negative               2105
Neutral                 590
Unknown                   9
Customer Experience       1
Name: count, dtype: int64

In [49]:
amazon_reviews_resolved.to_csv('outputs/amazon_reviews_with_topics_10000_resolved.csv', index=False)

In [2]:
amazon_reviews_resolved = pd.read_csv('outputs/amazon_reviews_with_topics_10000_resolved.csv')

In [52]:
amazon_reviews_resolved[amazon_reviews_resolved['Sentiment'] == 'Unknown']

,Reviews,Ratings,Topic,Sentiment
239,NaN,5.0,Unknown,Unknown
591,10 points,5.0,Unknown,Unknown
1496,A,5.0,Unknown,Unknown
1865,NaN,5.0,Unknown,Unknown
2207,NaN,5.0,Unknown,Unknown
3255,Nimrod,5.0,Unknown,Unknown
3639,Regular,4.0,Unknown,Unknown
4343,Bought this and a $5 Yamaha plastic recorder f...,5.0,Unknown,Unknown
9568,Hayden,5.0,Unknown,Unknown


### Checking accuracy of Sentiment

In [53]:
amazon_reviews_resolved['Sentiment'].value_counts()

Sentiment
Positive               7295
Negative               2105
Neutral                 590
Unknown                   9
Customer Experience       1
Name: count, dtype: int64

In [54]:
amazon_reviews_resolved['Ratings'].value_counts()

Ratings
5.0    6431
4.0    1580
1.0     807
3.0     741
2.0     441
Name: count, dtype: int64

In [55]:
# Create a function to map Sentiment to Rating categories
def map_sentiment_to_rating(sentiment):
    if sentiment == 'Positive':
        return [4, 5]
    elif sentiment == 'Negative':
        return [1, 2]
    elif sentiment == 'Neutral':
        return [3]
    else:
        return []

# Check if the sentiment matches the label
def is_correct_prediction(row):
    possible_ratings = map_sentiment_to_rating(row['Sentiment'])
    return row['Ratings'] in possible_ratings

# Apply the function to check correctness
amazon_reviews_resolved["correct_sentiment"] = amazon_reviews_resolved.apply(is_correct_prediction, axis=1)


In [64]:
# Create a function to map Sentiment to Rating categories
def map_sentiment_to_rating(sentiment):
    if sentiment == 'Positive':
        return [4, 5]
    elif sentiment == 'Negative':
        return [1, 2]
    elif sentiment == 'Neutral':
        return [3]
    else:
        return []

# Check if the sentiment matches the label
def is_correct_prediction(row):
    possible_ratings = map_sentiment_to_rating(row['Sentiment'])
    return row['Ratings'] in possible_ratings

def calculate_metrics(df, target_sentiment):
    """
    Calculate classification metrics for a specific sentiment class
    Returns: Dictionary containing TP, FP, FN, TN, Precision, Recall, F1 Score, and Accuracy
    """
    # Apply the function to check correctness if not already done
    if 'correct_sentiment' not in df.columns:
        df['correct_sentiment'] = df.apply(is_correct_prediction, axis=1)
    
    # True Positives: Sentiment is correctly predicted as the target class
    true_positive = len(df[
        (df['Sentiment'] == target_sentiment) & 
        (df['correct_sentiment'] == True)
    ])
    
    # False Positives: Sentiment is incorrectly predicted as the target class
    false_positive = len(df[
        (df['Sentiment'] == target_sentiment) & 
        (df['correct_sentiment'] == False)
    ])
    
    # False Negatives: Actual ratings fall in target sentiment range but different sentiment predicted
    target_ratings = map_sentiment_to_rating(target_sentiment)
    false_negative = len(df[
        (df['Ratings'].isin(target_ratings)) & 
        (df['Sentiment'] != target_sentiment)
    ])
    
    # True Negatives: Neither the ratings nor the predicted sentiment match the target class
    true_negative = len(df[
        (~df['Ratings'].isin(target_ratings)) & 
        (df['Sentiment'] != target_sentiment) & 
        (df['correct_sentiment'] == True)
    ])
    
    # Calculate additional metrics
    # Prevent division by zero
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
    
    return {
        'true_positive': true_positive,
        'false_positive': false_positive,
        'false_negative': false_negative,
        'true_negative': true_negative,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'accuracy': accuracy
    }

# Calculate and display metrics for each sentiment
for sentiment in ['Positive', 'Negative', 'Neutral']:
    metrics = calculate_metrics(amazon_reviews_resolved, sentiment)
    print(f"\n{sentiment} Sentiment:")
    print(f"True Positive: {metrics['true_positive']}")
    print(f"False Positive: {metrics['false_positive']}")
    print(f"False Negative: {metrics['false_negative']}")
    print(f"True Negative: {metrics['true_negative']}")
    print(f"Precision: {metrics['precision']:.3f}")
    print(f"Recall: {metrics['recall']:.3f}")
    print(f"F1 Score: {metrics['f1_score']:.3f}")
    print(f"Accuracy: {metrics['accuracy']:.3f}")


Positive Sentiment:
True Positive: 7202
False Positive: 93
False Negative: 809
True Negative: 1374
Precision: 0.987
Recall: 0.899
F1 Score: 0.941
Accuracy: 0.905

Negative Sentiment:
True Positive: 1231
False Positive: 874
False Negative: 17
True Negative: 7345
Precision: 0.585
Recall: 0.986
F1 Score: 0.734
Accuracy: 0.906

Neutral Sentiment:
True Positive: 143
False Positive: 447
False Negative: 598
True Negative: 8433
Precision: 0.242
Recall: 0.193
F1 Score: 0.215
Accuracy: 0.891


In [65]:
amazon_reviews_resolved['Topic'].value_counts()

Topic
Sound & Performance          2170
Functionality & Usability    1783
Durability                   1107
Product Quality              1046
Price & Value                 759
Aesthetics                    455
Usage Experience              420
User Experience               365
Customer Experience           313
Size & Portability            306
Technical Aspects             304
Compatibility                 245
Gift Giving                   201
Shipping & Delivery           190
Comfort & Design              154
Learning Curve                104
Target Audience                33
Emotional Response             18
Brand Perception               13
Unknown                         9
Customer Service                3
Sound Quality                   1
Battery Life                    1
Name: count, dtype: int64

### Checking Accuracy of predicted topics

In [68]:
N = 10000  # population size
z = 1.96   # z-score for 95% confidence
p = 0.5    # maximum variance
q = 1 - p  # 0.5
d = 0.05   # margin of error

n = (N * z**2 * p * q) / (d**2 * (N-1) + z**2 * p * q)

In [69]:
n

369.983704382866

This means that we need to check 370 rows manually to be 95% sure that our accuracy is close to real accuracy.

In [72]:
review_sample = amazon_reviews_resolved.sample(370, random_state=42)

In [74]:
review_sample.to_excel('outputs/amazon_reviews_sample_370.xlsx')

In [5]:
import pandas as pd
import numpy as np

def map_sentiment_to_rating(sentiment):
    if sentiment == 'Positive':
        return [4, 5]
    elif sentiment == 'Negative':
        return [1, 2]
    elif sentiment == 'Neutral':
        return [3]
    else:
        return []

def is_correct_prediction(row):
    possible_ratings = map_sentiment_to_rating(row['Sentiment'])
    return row['Ratings'] in possible_ratings

def calculate_metrics(df, target_sentiment):
    """
    Calculate classification metrics for a specific sentiment class
    Returns: Dictionary containing TP, FP, FN, TN, Precision, Recall, F1 Score, and Accuracy
    """
    # Apply the function to check correctness if not already done
    if 'correct_sentiment' not in df.columns:
        df['correct_sentiment'] = df.apply(is_correct_prediction, axis=1)
    
    # True Positives: Sentiment is correctly predicted as the target class
    true_positive = len(df[
        (df['Sentiment'] == target_sentiment) & 
        (df['correct_sentiment'] == True)
    ])
    
    # False Positives: Sentiment is incorrectly predicted as the target class
    false_positive = len(df[
        (df['Sentiment'] == target_sentiment) & 
        (df['correct_sentiment'] == False)
    ])
    
    # False Negatives: Actual ratings fall in target sentiment range but different sentiment predicted
    target_ratings = map_sentiment_to_rating(target_sentiment)
    false_negative = len(df[
        (df['Ratings'].isin(target_ratings)) & 
        (df['Sentiment'] != target_sentiment)
    ])
    
    # True Negatives: Neither the ratings nor the predicted sentiment match the target class
    true_negative = len(df[
        (~df['Ratings'].isin(target_ratings)) & 
        (df['Sentiment'] != target_sentiment) & 
        (df['correct_sentiment'] == True)
    ])
    
    # Calculate metrics with error handling
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
    
    return {
        'true_positive': true_positive,
        'false_positive': false_positive,
        'false_negative': false_negative,
        'true_negative': true_negative,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'accuracy': accuracy
    }

def print_all_metrics(df):
    """
    Calculate and display metrics for all sentiment classes and weighted averages
    """
    sentiments = ['Positive', 'Negative', 'Neutral']
    all_metrics = {}
    total_samples = len(df)
    
    # Calculate metrics for each sentiment
    for sentiment in sentiments:
        metrics = calculate_metrics(df, sentiment)
        all_metrics[sentiment] = metrics
        
        print(f"\n{sentiment} Sentiment Metrics:")
        print(f"True Positive: {metrics['true_positive']}")
        print(f"False Positive: {metrics['false_positive']}")
        print(f"False Negative: {metrics['false_negative']}")
        print(f"True Negative: {metrics['true_negative']}")
        print(f"Precision: {metrics['precision']:.3f}")
        print(f"Recall: {metrics['recall']:.3f}")
        print(f"F1 Score: {metrics['f1_score']:.3f}")
        print(f"Accuracy: {metrics['accuracy']:.3f}")
    
    # Calculate weighted averages
    class_weights = {sentiment: len(df[df['Sentiment'] == sentiment]) / total_samples 
                    for sentiment in sentiments}
    weighted_precision = sum(metrics['precision'] * class_weights[sentiment] 
                           for sentiment, metrics in all_metrics.items())
    weighted_recall = sum(metrics['recall'] * class_weights[sentiment] 
                         for sentiment, metrics in all_metrics.items())
    weighted_f1 = sum(metrics['f1_score'] * class_weights[sentiment] 
                     for sentiment, metrics in all_metrics.items())
    weighted_accuracy = sum(metrics['accuracy'] * class_weights[sentiment] 
                          for sentiment, metrics in all_metrics.items())
    
    print("\nWeighted Metrics:")
    print(f"Weighted Precision: {weighted_precision:.3f}")
    print(f"Weighted Recall: {weighted_recall:.3f}") 
    print(f"Weighted F1 Score: {weighted_f1:.3f}")
    print(f"Weighted Accuracy: {weighted_accuracy:.3f}")
    
    return all_metrics

# Usage example:
metrics = print_all_metrics(amazon_reviews_resolved)


Positive Sentiment Metrics:
True Positive: 7202
False Positive: 93
False Negative: 809
True Negative: 1374
Precision: 0.987
Recall: 0.899
F1 Score: 0.941
Accuracy: 0.905

Negative Sentiment Metrics:
True Positive: 1231
False Positive: 874
False Negative: 17
True Negative: 7345
Precision: 0.585
Recall: 0.986
F1 Score: 0.734
Accuracy: 0.906

Neutral Sentiment Metrics:
True Positive: 143
False Positive: 447
False Negative: 598
True Negative: 8433
Precision: 0.242
Recall: 0.193
F1 Score: 0.215
Accuracy: 0.891

Weighted Metrics:
Weighted Precision: 0.858
Weighted Recall: 0.875
Weighted F1 Score: 0.854
Weighted Accuracy: 0.903
